In [15]:
import pandas as pd
import numpy as np

In [16]:
# Load the data
df = pd.read_pickle('shared/Project-3_NYC_311_Calls.pkl')


Due to the kernel memory size, we will find a way to deal with millions of data entries. The method here is to optimize data types for reducing memory usage.

In [18]:
# Optimize numerical columns
for col in df.select_dtypes(include=['int', 'float']).columns:
    df[col] = pd.to_numeric(df[col], downcast='integer')

# Convert object columns to category if they have a limited set of values
for col in df.select_dtypes(include=['object']).columns:
    if df[col].nunique() / len(df[col]) < 0.5:
        df[col] = df[col].astype('category')


In [3]:
# Display the first few rows
print(df.head())

   Unique Key        Created Date Agency  \
0    20184537 2011-04-06 00:00:00    HPD   
1    20184538 2011-04-06 00:00:00    HPD   
2    20184539 2011-04-06 00:00:00    HPD   
3    54732265 2022-07-08 11:14:43   DSNY   
4    20184540 2011-04-06 00:00:00    HPD   

                                         Agency Name        Complaint Type  \
0  Department of Housing Preservation and Develop...               HEATING   
1  Department of Housing Preservation and Develop...  GENERAL CONSTRUCTION   
2  Department of Housing Preservation and Develop...       PAINT - PLASTER   
3                           Department of Sanitation       Dirty Condition   
4  Department of Housing Preservation and Develop...              NONCONST   

  Descriptor         Location Type Incident Zip      City  \
0       HEAT  RESIDENTIAL BUILDING      10002.0  NEW YORK   
1    WINDOWS  RESIDENTIAL BUILDING      11236.0  BROOKLYN   
2      WALLS  RESIDENTIAL BUILDING      10460.0     BRONX   
3      Trash          

In [4]:
# General information about the dataset
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33780977 entries, 0 to 33780976
Data columns (total 12 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   Unique Key              int64         
 1   Created Date            datetime64[ns]
 2   Agency                  object        
 3   Agency Name             object        
 4   Complaint Type          object        
 5   Descriptor              object        
 6   Location Type           object        
 7   Incident Zip            object        
 8   City                    object        
 9   Resolution Description  object        
 10  Borough                 object        
 11  Open Data Channel Type  object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 3.0+ GB
None


In [5]:
# Statistical summary
print(df.describe())

         Unique Key                   Created Date
count  3.378098e+07                       33780977
mean   3.802665e+07  2017-06-24 23:32:20.293824512
min    1.056422e+07            2010-01-01 00:00:00
25%    2.793121e+07            2014-04-08 10:35:34
50%    3.798303e+07            2017-11-14 12:46:13
75%    4.830951e+07            2020-12-02 07:49:48
max    5.840682e+07            2023-08-04 12:00:00
std    1.209574e+07                            NaN


In [6]:
# Checking for missing values
print(df.isnull().sum())

Unique Key                      0
Created Date                    0
Agency                          0
Agency Name                     0
Complaint Type                  0
Descriptor                 586677
Location Type             7140574
Incident Zip              1507958
City                      1981664
Resolution Description    1254890
Borough                     47074
Open Data Channel Type          0
dtype: int64


In [4]:
# Fill missing values
df.fillna('unknown', inplace=True)

In [8]:
# Checking for missing values
print(df.isnull().sum())

Unique Key                0
Created Date              0
Agency                    0
Agency Name               0
Complaint Type            0
Descriptor                0
Location Type             0
Incident Zip              0
City                      0
Resolution Description    0
Borough                   0
Open Data Channel Type    0
dtype: int64


In [20]:
# Convert 'Created Date' to datetime and set as index
df['Created Date'] = pd.to_datetime(df['Created Date'])
df = df.set_index(df['Created Date'])
del df['Created Date']


What is the average number of daily complaints received in 2022?

In [21]:
# Filter data for the year 2022
df_2022 = df['2022-01-01':'2022-12-31']

# Resample by day and count complaints
daily_complaints_2022 = df_2022.resample('D').count()

# Calculate the average number of daily complaints
average_daily_complaints = daily_complaints_2022['Unique Key'].mean()
average_daily_complaints


8684.320547945206

On which single date were the maximum number of calls received?

In [22]:
# Ensure 'Created Date' is the index and in datetime format if it's not already
if not isinstance(df.index, pd.DatetimeIndex):
    df['Created Date'] = pd.to_datetime(df['Created Date'])
    df = df.set_index('Created Date')

# Resample by day and count complaints
daily_complaints = df.resample('D').count()

# Identify the date with the maximum number of complaints
max_complaints_date = daily_complaints['Unique Key'].idxmax()
max_complaints_date.date()

datetime.date(2020, 8, 4)

On the date the maximum number of calls were received, what was the most important complaint type?

In [24]:
# Convert index to DatetimeIndex if it's not already
if not isinstance(df.index, pd.DatetimeIndex):
    df.index = pd.to_datetime(df.index)

# Normalize the max_complaints_date to remove time component
max_complaints_date = pd.to_datetime(max_complaints_date).normalize()

# Filter data for the max complaints date
data_on_max_date = df[df.index.normalize() == max_complaints_date]

# Find the most common complaint type
most_common_complaint = data_on_max_date['Complaint Type'].value_counts().idxmax()
print(f"The most common complaint on {max_complaints_date.date()} was: {most_common_complaint}")


The most common complaint on 2020-08-04 was: Damaged Tree


Quietest month: Group the data by months, and identify the month that historically has the fewest number of calls.

In [31]:
# Group the data by month
df_monthly = df.groupby(df.index.to_period('M')).size()
# Find the fewest number of calls month
quietest_month = df_monthly.idxmin().strftime('%b')
quietest_month

'Aug'

Resample your time series to a daily frequency.  Perform ETS decomposition based on an additive model.  What is the value of the seasonal component on 2020-12-25 (rounded to the nearest integer)?

In [32]:
import statsmodels.api as sm

# Ensure the DataFrame index is in datetime format
if not isinstance(df.index, pd.DatetimeIndex):
    df.index = pd.to_datetime(df.index)

# Resample to daily frequency and count calls
daily_calls = df.resample('D').size()

# Perform ETS decomposition
decomposition = sm.tsa.seasonal_decompose(daily_calls, model='additive')

# Extract the seasonal component
seasonal_component = decomposition.seasonal

# Find the value for 2020-12-25
seasonal_value = seasonal_component.loc['2020-12-25']
rounded_value = round(seasonal_value)

rounded_value


183

Calculate the autocorrelation of the number of daily calls with the number of calls the day prior, ie lag of 1.  (Use the daily series).

In [33]:
# Calculate autocorrelation for lag of 1
autocorrelation = daily_calls.autocorr(lag=1)
autocorrelation


0.7517059728398577

Forecast the daily series with a test set of 90 days using the Prophet library.  What is your RMSE on your test set?

In [34]:
from prophet import Prophet
from sklearn.metrics import mean_squared_error

# Prepare data for Prophet
df_prophet = daily_calls.reset_index()
df_prophet.columns = ['ds', 'y']

# Split into training and test set (last 90 days as test set)
train = df_prophet.iloc[:-90]
test = df_prophet.iloc[-90:]

# Create and fit the Prophet model
model = Prophet()
model.fit(train)

# Make predictions on the test set
future = test.drop(columns='y')
forecast = model.predict(future)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(test['y'], forecast['yhat']))
rmse


15:56:39 - cmdstanpy - INFO - Chain [1] start processing
15:56:40 - cmdstanpy - INFO - Chain [1] done processing


1231.513760758433